In [14]:
from import_functions_generic import *

# Functions

In [16]:

def compute_im_rad_grid(im):
    '''
    Compute a 2D radius grid with the same shape than the input image 'im'.
    '''
    nx = np.shape(im)[1]
    ny = np.shape(im)[0]
    x,y = np.linspace(-nx//2,nx//2,nx), np.linspace(-ny//2,ny//2,ny)
    xs,ys = np.meshgrid(x, y, sparse=True)
    zs = np.sqrt(xs**2 + ys**2)
    return zs


In [17]:

def compute_mean_map_ann(im, dr, alpha=0, add_input_im_rad=0, im_rad=None, display=0):
    '''
    Compute the averaged mean 2D map by annular estimation with slippery annulus.
    Use the function compute im_rad_grid() to derive the distance (in pixel) of each
    pixel to the center of the image.

    Inputs:
        .'im' (2D-array): image
        .'dr' (float): width of the annulus for which the detection limit is computed
        (optional)
        .'alpha' (float): factor to consider bigger annulus to derive the noise
            Goal: have smoother transition between annuli
        .'add_input_im_rad' (boolean): input corresponding to the 2D radius grid provided
            (if parameter set to 1) or not (parameter set to 0, default value)
        .'im_rad' (2D-array): None (if add_input_im_rad == 0) or a 2D radius grid with the
            same shape than the input image 'im' (if add_input_im_rad == 1)
        .'display' (boolean):
            Default value: 0 (False) i.e. do not display details/information

    Output:
        .'im_noise' (2D-array): detection limit map
    '''
    t0 = time.time()
    if display: print("\nComputing the standard deviation limit map by using the 1D-annulus method")
    h, w = np.shape(im)
    noise_tot, noise_ann = [], []

    im_noise, im_nan, = np.zeros((h,w)), np.empty((h,w))
    if add_input_im_rad : im_radius_grid = im_rad
    else : im_radius_grid = compute_im_rad_grid(im)

    im_nan[:] = np.NaN
    rad, x0,y0 = 0, w//2+1, h//2+1

    # Until the annulus is smaller than the size of the image
    while rad < w//2 * 1.45 : # 1.45 slightly bigger than sqrt(2) to be sure to cover all the field of view and not only a circle of radius r
        # One annulus is considered
        cond_annulus_large = np.logical_and(im_radius_grid >= rad-alpha, rad + dr + alpha >= im_radius_grid)
        cond_annulus_thin  = np.logical_and(im_radius_grid >= rad, rad + dr >= im_radius_grid)
        im_annulus = np.where(cond_annulus_large, im, im_nan)
        # the noise over the annulus is computed
        noise_annulus = np.nanmean(im_annulus)
        # and the image is set at this noise for this given annulus
        im_noise[cond_annulus_thin] = noise_annulus
        rad += dr
    if display: print("-> took {} seconds".format(time.time()-t0))
    return im_noise


def compute_mad_map_ann(im, dr, alpha=0, add_input_im_rad=0, im_rad=None, display=0):
    '''
    Compute the mean absolute deviation 2D map by annular noise estimation with slippery annulus.
    Use the function compute im_rad_grid() to derive the distance (in pixel) of each
    pixel to the center of the image.

    Inputs:
        .'im' (2D-array): image
        .'dr' (float): width of the annulus for which the detection limit is computed
        (optional)
        .'alpha' (float): factor to consider bigger annulus to derive the noise
            Goal: have smoother transition between annuli
        .'add_input_im_rad' (boolean): input corresponding to the 2D radius grid provided
            (if parameter set to 1) or not (parameter set to 0, default value)
        .'im_rad' (2D-array): None (if add_input_im_rad == 0) or a 2D radius grid with the
            same shape than the input image 'im' (if add_input_im_rad == 1)
        .'display' (boolean):
            Default value: 0 (False) i.e. do not display details/information

    Output:
        .'im_noise' (2D-array): detection limit map
    '''
    t0 = time.time()
    if display: print("\nComputing the standard deviation limit map by using the 1D-annulus method")
    h, w = np.shape(im)
    noise_tot, noise_ann = [], []

    im_noise, im_nan, = np.zeros((h,w)), np.empty((h,w))
    if add_input_im_rad : im_radius_grid = im_rad
    else : im_radius_grid = compute_im_rad_grid(im)

    im_nan[:] = np.NaN
    rad, x0,y0 = 0, w//2+1, h//2+1

    # Until the annulus is smaller than the size of the image
    while rad < w//2 * 1.45 : # 1.45 slightly bigger than sqrt(2) to be sure to cover all the field of view and not only a circle of radius r
        # One annulus is considered
        cond_annulus_large = np.logical_and(im_radius_grid >= rad-alpha, rad + dr + alpha >= im_radius_grid)
        cond_annulus_thin  = np.logical_and(im_radius_grid >= rad, rad + dr >= im_radius_grid)
        im_annulus = np.where(cond_annulus_large, im, im_nan)
        # the noise over the annulus is computed
        noise_annulus = molmap.compute_mad(im_annulus)
        # and the image is set at this noise for this given annulus
        im_noise[cond_annulus_thin] = noise_annulus
        rad += dr
    if display: print("-> took {:.3f} seconds.".format(time.time()-t0))
    return im_noise


def compute_std_map_ann(im, dr, alpha=0, add_input_im_rad=0, im_rad=None, display=0):
    '''
    Compute the standard deviation 2D map by annular noise estimation with slippery annulus.
    Use the function compute im_rad_grid() to derive the distance (in pixel) of each
    pixel to the center of the image.

    Inputs:
        .'im' (2D-array): image
        .'dr' (float): width of the annulus for which the detection limit is computed
        (optional)
        .'alpha' (float): factor to consider bigger annulus to derive the noise
            Goal: have smoother transition between annuli
        .'add_input_im_rad' (boolean): input corresponding to the 2D radius grid provided
            (if parameter set to 1) or not (parameter set to 0, default value)
        .'im_rad' (2D-array): None (if add_input_im_rad == 0) or a 2D radius grid with the
            same shape than the input image 'im' (if add_input_im_rad == 1)
        .'display' (boolean):
            Default value: 0 (False) i.e. do not display details/information

    Output:
        .'im_noise' (2D-array): detection limit map
    '''
    t0 = time.time()
    if display: print("\nComputing the standard deviation limit map by using the 1D-annulus method")
    h, w = np.shape(im)
    noise_tot, noise_ann = [], []

    im_noise, im_nan, = np.zeros((h,w)), np.empty((h,w))
    if add_input_im_rad : im_radius_grid = im_rad
    else : im_radius_grid = compute_im_rad_grid(im)

    im_nan[:] = np.NaN
    rad, x0,y0 = 0, w//2+1, h//2+1

    # Until the annulus is smaller than the size of the image
    while rad < w//2 * 1.45 : # 1.45 slightly bigger than sqrt(2) to be sure to cover all the field of view and not only a circle of radius r
        # One annulus is considered
        cond_annulus_large = np.logical_and(im_radius_grid >= rad-alpha, rad + dr + alpha >= im_radius_grid)
        cond_annulus_thin  = np.logical_and(im_radius_grid >= rad, rad + dr >= im_radius_grid)
        im_annulus = np.where(cond_annulus_large, im, im_nan)
        # the noise over the annulus is computed
        noise_annulus = np.nanstd(im_annulus)
        # and the image is set at this noise for this given annulus
        im_noise[cond_annulus_thin] = noise_annulus
        rad += dr
    if display: print("-> took {} seconds".format(time.time()-t0))
    return im_noise

In [18]:
def rad_profile(im, center, mode='std', pixscale=1, dr=1, skip_pix=None):
    '''
    Assumption: center at n//2-1 i.e. between the four central pixel (for even image)

    Function useful to reproduce the Figure 3 from Hoeijmakers et al. (2018).
    '''
    if skip_pix == None: skip_pix=3

    if mode == 'std':
        im = compute_std_map_ann(im,dr=dr)
        #profile = (im_std[skip_pix+int(center):,int(center)]+im_std[skip_pix+int(center):,int(center+1)])/2
        profile = im[skip_pix+int(center):,int(center)]
        separations = np.arange(skip_pix,len(im)//2+1.,1)

    elif mode == 'mad':
        im = compute_mad_map_ann(im,dr=dr)
        #profile = (im_std[skip_pix+int(center):,int(center)]+im_std[skip_pix+int(center):,int(center+1)])/2
        profile = im[skip_pix+int(center):,int(center)]
        separations = np.arange(skip_pix,len(im)//2+1.,1)

    elif mode == 'mean':
        im = compute_mean_map_ann(im,dr=dr)
        profile = im[skip_pix+int(center):,int(center)]
        separations = np.arange(skip_pix,len(im)//2+1.,1)
    else: print('Precise a mode available, e.g. std or mean. Here the mode given is {}'.format(mode))

    return profile, separations*pixscale

In [19]:
fn = '/Users/desgranc/Documents/work/projects/HD120326/data/reduced/IRDAP/2018-06-01/reduced_pdi/star_pol_subtr/HIP67497_2018-06-02_U_phi_star_pol_subtr.fits'
f = fits.getdata(fn)

cropping = 450
f = f[cropping:-cropping, cropping:-cropping]
print(np.shape(f))


platescale = 12.25

(124, 124)


In [20]:
# Consider one image
im = f
# Define its center
center = np.shape(im)[0]//2#-0.5
print('The center of the image is ({},{}).'.format(center,center))

# Consider the radial profile
profile_std,  separations = rad_profile(im, center, mode='std', pixscale=platescale*1e-3)
profile_mean, separations = rad_profile(im, center, mode='mean', pixscale=platescale*1e-3)

The center of the image is (62,62).


/Users/desgranc/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1670: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/var/folders/k2/sfv4r4z938l77n1t4ydth2v401phl2/T/ipykernel_31763/1242410239.py:42: RuntimeWarning: Mean of empty slice
  noise_annulus = np.nanmean(im_annulus)
